In [21]:
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext, LLMPredictor
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import GPTVectorStoreIndex
from llama_index.vector_stores import PineconeVectorStore
import os
import pandas as pd
import pinecone
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# GT
qa = pd.read_csv('./data/gene_qa.csv', encoding='ISO-8859-1')
qa = qa[['question','answer']]

# external data
gene = pd.read_csv('./data/gene_revised.csv')
data = gene['description'].tolist()

# transform doc and node parsing to make vector 
docs = []
for i, d in enumerate(data):
    doc = Document(text = d)
    docs.append(doc)

parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(docs)

# pinecone setting & upsert

# snu 계정 pinecone api_key
PINECONE_API_KEY = '6e7208af-6071-4493-854f-4ea43a3c5d53'
PINECONE_ENV = 'gcp-starter'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# make pinecone index
index_name = 'gene-index'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric='cosine'
    )

pinecone_index = pinecone.GRPCIndex(index_name)

# connect vector store to index
vector_store = PineconeVectorStore(pinecone_index = pinecone_index)
storage_context = StorageContext.from_defaults(
    vector_store = vector_store
)

# embedding
model_name = "BAAI/bge-base-en-v1.5"
embed_model = HuggingFaceEmbedding(model_name=model_name,embed_batch_size=100)
service_context = ServiceContext.from_defaults(embed_model=embed_model) # default: llama2-13b-chat
### using local model 
# llm_predictor = LLMPredictor(model) 이걸로 로컬모델 연동가능한 것 같습니다. 
# service_context = ServiceContext.from_defaults(llm_predictor = llm_predictor, embed_model=embed_model) 

# nodes: Sequence[Node]
index = GPTVectorStoreIndex.from_documents(
    docs, storage_context=storage_context,
    service_context=service_context
)

cuda
******
Could not load OpenAI model. Using default LlamaCPP=llama2-13b-chat. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******
total size (MB): 7365.83


7025it [10:36, 11.03it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:       

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB (GPU 0; 10.91 GiB total capacity; 9.99 GiB already allocated; 43.06 MiB free; 10.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
import pandas as pd
gene = pd.read_csv('./data/gene_revised.csv')
data = gene['description'].tolist()
data[0]

'The AAAS gene provides instructions for making a protein called ALADIN whose function is not well understood. Within cells, ALADIN is found in the nuclear envelope, the structure that surrounds the nucleus and separates it from the rest of the cell. Based on its location, ALADIN is thought to be involved in the movement of molecules into and out of the nucleus.'

In [ ]:
# gene_data load code
# import json
# from langchain.document_loaders import WebBaseLoader

# with open('gene_url.json', 'r') as file:
#    data = json.load(file)
# web_links = data
# loader = WebBaseLoader(web_links)
# documents = loader.load()

# #to save loaded documents to csv file
# import csv

# with open('gene.csv', 'w', newline='') as file:
#    writer = csv.writer(file)
#    writer.writerow(['col'])
#    for item in documents:
#        writer.writerow([item])
# gene = pd.read_csv('./gene.csv')
# data = gene['col'].tolist()

In [68]:
# gene data preprocessing code

# gene = pd.read_csv('./gene.csv')
# data = gene['col'].tolist()

# import re

# revises = []

# for i, d in enumerate(data):
#     try:
#         start_idx = re.search(r'\\nThe',d).start()+2
#         revise = d[start_idx:]
#         end_idx =re.search(r'\\n\\n\\n\\nHealth',revise).start()
#         revise = revise[:end_idx]
#         revises.append([revise])
#     except:
#         start_idx = re.search(r'description',e).end() + 3
#         r_e = e[start_idx:]
#         end_idx = re.search(r'Learn',r_e).start()
#         revise = r_e[:end_idx]
#         revises.append([revise])


# print(revises)

# # save to csv

# import csv

# fields = ['description']

# with open('gene_revised.csv', 'w', newline='') as csv_file:
#     writer = csv.writer(csv_file)
#     writer.writerow(fields)
#     writer.writerows(revises)

In [3]:
import pandas as pd
qa = pd.read_csv('./gene_qa.csv', encoding='ISO-8859-1')
qa = qa[['question','answer']]
qa

,question,answer
0,"Acrodermatitis enteropathica, Birk-Landau-Pere...",Zinc
1,Which of the following would represent a frame...,c.169_170insA
2,"On examination, a patient is noted to have ¡°c...",Urine glycosaminoglycans
3,"A man and a woman have 2 children, both of who...",12.50%
4,An infant was slightly premature and is being ...,Williams syndrome
...,...,...
79,The wings of a bat and a bird have similar for...,Analogous
80,You are analyzing whole genome sequencing data...,Cornelia de Lange syndrome
81,A teen-ager has a several year history of inte...,Dentatorubral-pallidoluysian atrophy (DRPLA)
82,In general (and with the caveat that there can...,Dravet syndrome


In [8]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(docs)

In [13]:
query_engine = index.as_query_engine()
response = query_engine.query("What is ABCA12 gene? if you refer document, please answer with reference part")
print(response)

The ABCA12 gene provides instructions for making a protein known as an ATP-binding cassette (ABC) transporter. This information is mentioned in the document.


In [ ]:
index = SummaryIndex.from_documents(docs)

query_engine = index.as_query_engine()
response = query_engine.query("What is ABCA12 gene? ")
display(Markdown(f"<b>{response}</b>"))